# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. **(A to już znamy!!!)**

## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [1]:
import random
import math

In [2]:
def prime_naive(n):
    if n < 2:
        return False

    m = math.isqrt(n)
    for i in range(2, m + 1):
        if n % i == 0:
            return False

    return True

In [3]:
def prime_miller_rabin(n, k=1):
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [4]:
primes_less_than_2000 = [x for x in range(2, 2000) if prime_naive(x)]

#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    #napisz swój kod tutaj
    result = False

    while not result:
        result2 = True
        p = random.getrandbits(keysize)
        p |= (1 << 0)
        p |= (1 << (keysize - 1))

        for prime in primes_less_than_2000:
            if p % prime == 0 and p != prime:
                result2 = False
                break
        
        if not result2: 
            continue
        
        result = prime_miller_rabin(p, 5)
    return p

In [11]:
print(generatePrime(128))

278004168549185071599169501810856608661


### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$ **(potrzebujemy rozszerzonego algorytmu Euklidesa!!!)**
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [14]:
def extended_euclidean(a: int, b: int):
    r_prev, r = a, b
    s_prev, s = 1, 0
    t_prev, t = 0, 1

    while r != 0:
        q = r_prev // r

        r_prev, r = r, r_prev - q * r
        s_prev, s = s, s_prev - q * s
        t_prev, t = t, t_prev - q * t

    return r_prev, s_prev, t_prev

In [13]:
def mod_inverse(a: int, n: int):
    g, s, t = extended_euclidean(a, n)
    if g != 1:
        return None
    else:
        return s % n

In [28]:
import random, os

def generateKey(keySize):
    #napisz swój kod tutaj
    p = generatePrime(keySize//2)
    q = generatePrime(keySize//2)
    
    while p == q:
        q = generatePrime(keySize//2)
    
    n = p * q
    phi = (p - 1) * (q - 1)

    e = None
    while True:
        cand = random.randrange(3, phi - 1, 2)
        if extended_euclidean(cand, phi)[0] == 1:
            e = cand
            break

    publicKey = (n, e)

    d = mod_inverse(e, phi)
    privateKey = (n, d)

    print(f'Klucz publiczny (n, e): {publicKey}')
    print(f'Klucz prywatny (n, d): {privateKey}')
    
    return (publicKey, privateKey)

def makeKeyFiles(keySize):
    public, private = generateKey(keySize)
    n_pub, e = public
    n_priv, d = private

    os.makedirs('keys', exist_ok=True)

    pub_path = os.path.join('keys', 'publicKey.txt')
    priv_path = os.path.join('keys', 'privateKey.txt')

    with open(pub_path, 'w') as f:
        f.write(f'{n_pub}\n{e}\n')

    with open(priv_path, 'w') as f:
        f.write(f'{n_priv}\n{d}\n')

    print(f'Zapisano klucz publiczny do: {pub_path}')
    print(f'Zapisano klucz prywatny do: {priv_path}')
  
makeKeyFiles(16)

Klucz publiczny (n, e): (22879, 20731)
Klucz prywatny (n, d): (22879, 13411)
Zapisano klucz publiczny do: keys\publicKey.txt
Zapisano klucz prywatny do: keys\privateKey.txt


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [22]:
def encrypt(message, modulus, exp):
    # kod szyfrowania     
    message_encrypted = [pow(ord(char), exp, modulus) for char in message]
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [23]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = [chr(pow(char_code, exp, modulus)) for char_code in message_encrypted]
    return ('').join(message_ascii)

In [29]:
message = "HELLO RSA"
print(f'Wiadomość oryginalna: {message}')

publicKeyFile = os.path.join('keys', 'publicKey.txt')
with open(publicKeyFile, 'r') as f:
    n = int(f.readline().strip())
    e = int(f.readline().strip())
publicKey = (n, e)

privateKeyFile = os.path.join('keys', 'privateKey.txt')
with open(privateKeyFile, 'r') as f:
    n = int(f.readline().strip())
    d = int(f.readline().strip())
privateKey = (n, d)

message_encrypted = encrypt(message, publicKey[0], publicKey[1])
print(f'Zaszyfrowana wiadomość: {message_encrypted}')

message_decrypted = decrypt(message_encrypted, privateKey[0], privateKey[1])
print(f'Odszyfrowana wiadomość: {message_decrypted}')

Wiadomość oryginalna: HELLO RSA
Zaszyfrowana wiadomość: [1493, 16678, 12223, 12223, 22078, 3934, 9369, 11281, 14673]
Odszyfrowana wiadomość: HELLO RSA


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy) - zweryfikuj jak na wydajnosć wpływa zastosowanie różnych sposobów potęgowania dostępnych w Python i jego bibliotekach.  
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [ ]:

def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m